# Test for the Modified Number Operator
## Construct the Hamiltonians for 1d lattice

In [1]:
import sys
sys.path.append("../")
import fh_comm as fhc
import numpy as np
import pandas as pd

In [2]:
v = -1
u = 1
translatt = fhc.SubLattice(np.array([[2]]))
h0 = fhc.SumOp([fhc.HoppingOp(( 0,), ( 1,), s, v) for s in [0, 1]])
h1 = fhc.SumOp([fhc.HoppingOp((-1,), ( 0,), s, v) for s in [0, 1]])
h2 = fhc.SumOp([fhc.ProductOp([fhc.ModifiedNumOp((x,), s, 1) for s in [0, 1]], u) for x in [0, 1]])
h2_num = [[fhc.NumberOp((x,),s,1.) for s in [0,1]] for x in [0,1]]

In [3]:
# evaluate bound for 3 Hamiltonian terms
comm_bound_terms = fhc.commutator_bound_strang(3)
for cbt in comm_bound_terms:
    print(cbt)

1/24 * [H_0, [H_1, H_0]]
1/12 * [H_1, [H_1, H_0]]
1/12 * [H_2, [H_1, H_0]]
1/24 * [H_0, [H_2, H_0]]
1/12 * [H_1, [H_2, H_0]]
1/12 * [H_2, [H_2, H_0]]
1/24 * [H_1, [H_2, H_1]]
1/12 * [H_2, [H_2, H_1]]


Notice: Because we just modified H_2, than we just need to compute the term with H_2,
s.t. [H_2, [H_1, H_0]], [H_0, [H_2, H_0]], [H_1, [H_2, H_0]], [H_2, [H_2, H_0]], [H_1, [H_2, H_1]], [H_2, [H_2, H_1]]

In [4]:
# results of the first level commutator
result10 = fhc.commutator(h1,h0)
result20 = fhc.commutator(h2,h0)
result21 = fhc.commutator(h2,h1)

In [5]:
# results of the 6 terms with H_2
result = []
result.append(fhc.commutator(h2,result10))
result.append(fhc.commutator(h0,result20))
result.append(fhc.commutator(h1,result20))
result.append(fhc.commutator(h2,result20))
result.append(fhc.commutator(h1,result21))
result.append(fhc.commutator(h2,result21))
for i in range(len(result)):
    result[i] = fhc.simplify(result[i])

In [6]:
# for convinience, we construct a function to compute the modified number operator by hand
def theretical_h2_form(a: fhc.HamiltonianOp)->fhc.HamiltonianOp:
    return fhc.SumOp([fhc.commutator(fhc.ProductOp([h2_num[0][0], h2_num[0][1]],u),a), fhc.commutator(fhc.ProductOp([h2_num[1][0], h2_num[1][1]],u),a), 0.5*u*fhc.commutator(a, fhc.SumOp([Op for sublist in h2_num for Op in sublist]))])

In [7]:
# we compute the results by hand, and then compare the theoretical results with those above
the_result = []
the_result20 = theretical_h2_form(h0)
the_result21 = theretical_h2_form(h1)
the_result.append(theretical_h2_form(result10))
the_result.append(fhc.commutator(h0, the_result20))
the_result.append(fhc.commutator(h1, the_result20))
the_result.append(theretical_h2_form(the_result20))
the_result.append(fhc.commutator(h1, the_result21))
the_result.append(theretical_h2_form(the_result21))
for i in range(len(the_result)):
    the_result[i] = fhc.simplify(the_result[i])

In [8]:
# to check whether the 6 terms are the same, the outpus should be all 'True'
for i in range(len(result)):
    print(np.array_equal(result[i].as_field_operator().as_matrix((3,)).toarray(), the_result[i].as_field_operator().as_matrix((3,)).toarray()))

True
True
True
True
True
True


Here, we could try to construct the higher order suzuki, and try to print out the err bound.
Firstly, we need to reproduce the results in the paper.

In [3]:
h2_origin = fhc.SumOp([fhc.ProductOp([fhc.NumberOp((x,), s, 1) for s in [0, 1]], u) for x in [0, 1]])

In [5]:
hlist_origin = [h0,h1,h2_origin]
translatt = fhc.SubLattice([[2]])
comm_tab_origin = fhc.NestedCommutatorTable(hlist_origin, 5, translatt)
tab4_origin = comm_tab_origin.table(4)
err_bound_origin = 5 * [0]
method = fhc.SplittingMethod.suzuki(len(hlist_origin), 2)
s = (method.num_layers + 1) // 2
comm_bound_terms = fhc.commutator_bound(method, s)

To run the code below, we could obtain the same result as E.q.42 at Page7

In [5]:
data1 = {'Index0':[],
         'Index1':[],
         'Index2':[],
         'Index3':[],
         'Index4':[],
         'Err_term':[],
         'Err':[]}
for term in comm_bound_terms:
    num_int = sum(1 if i == len(hlist_origin)-1 else 0 for i in term.commidx)
    temp_err = term.weight * tab4_origin[term.commidx[0]][term.commidx[1]][term.commidx[2]][term.commidx[3]][term.commidx[4]].norm_bound()
    err_bound_origin[num_int] += temp_err
    data1['Index0'].append(term.commidx[0])
    data1['Index1'].append(term.commidx[1])
    data1['Index2'].append(term.commidx[2])
    data1['Index3'].append(term.commidx[3])
    data1['Index4'].append(term.commidx[4])
    data1['Err_term'].append(num_int)
    data1['Err'].append(temp_err)

In [6]:
df1 = pd.DataFrame(data1)
file_path = 'data1.xlsx'
df1.to_excel(file_path, index=False)

In [6]:
print(tab4_origin[1][2][1][1][1])

(-64) (g_{(-1,), (0,), up}) @ (g_{(-1,), (0,), dn}) + (-64) (n_{(-1,), up}) @ (n_{(-1,), dn}) + (64) (n_{(-1,), up}) @ (n_{(0,), dn}) + (64) (n_{(0,), up}) @ (n_{(-1,), dn}) + (-64) (n_{(0,), up}) @ (n_{(0,), dn})


In [7]:
print("err_bound_origin:", err_bound_origin)

err_bound_origin: [2.680944765044714, 17.646578262420086, 4.788947718065406, 0.8273585066999247, 0.12001862556075223]


In [7]:
hlist = [h0,h1,h2]
comm_tab = fhc.NestedCommutatorTable(hlist, 5, translatt)
tab4 = comm_tab.table(4)
err_bound = 5 * [0]

In [9]:
data2 = {'Index0':[],
         'Index1':[],
         'Index2':[],
         'Index3':[],
         'Index4':[],
         'Err_term':[],
         'Err':[]}
for term in comm_bound_terms:
    num_int = sum(1 if i == len(hlist)-1 else 0 for i in term.commidx)
    temp_err = term.weight * tab4[term.commidx[0]][term.commidx[1]][term.commidx[2]][term.commidx[3]][term.commidx[4]].norm_bound()
    err_bound[num_int] += temp_err
    data2['Index0'].append(term.commidx[0])
    data2['Index1'].append(term.commidx[1])
    data2['Index2'].append(term.commidx[2])
    data2['Index3'].append(term.commidx[3])
    data2['Index4'].append(term.commidx[4])
    data2['Err_term'].append(num_int)
    data2['Err'].append(temp_err)

In [ ]:
df2 = pd.DataFrame(data2)
file_path = 'data2.xlsx'
df2.to_excel(file_path, index=False)

In [8]:
print(tab4[1][2][1][1][1])

(-64) (g_{(-1,), (0,), up}) @ (g_{(-1,), (0,), dn}) + (-48) (n_{(-1,), up}) @ (n_{(-1,), dn}) + (48) (n_{(-1,), up}) @ (n_{(0,), dn}) + (8) (n_{(-1,), up}) @ (Mn_{(0,), dn}) + (48) (n_{(0,), up}) @ (n_{(-1,), dn}) + (-48) (n_{(0,), up}) @ (n_{(0,), dn}) + (8) (n_{(0,), up}) @ (Mn_{(-1,), dn}) + (-8) (n_{(0,), up}) @ (Mn_{(0,), dn}) + (-8) (n_{(1,), up}) @ (Mn_{(1,), dn}) + (8) (n_{(-1,), dn}) @ (Mn_{(0,), up}) + (8) (n_{(0,), dn}) @ (Mn_{(-1,), up}) + (-8) (n_{(0,), dn}) @ (Mn_{(0,), up}) + (-8) (n_{(1,), dn}) @ (Mn_{(1,), up})


In [5]:
commute_list1 = [h1,h2_origin,h1,h1,h1]
temp_result = commute_list1[0]
for i in range(1,len(commute_list1)):
    temp_result = fhc.commutator(commute_list1[i],temp_result)
print(fhc.simplify(temp_result))

(-32) (g_{(-1,), (0,), up}) @ (g_{(-1,), (0,), dn}) + (-24) (n_{(-1,), up}) @ (n_{(-1,), dn}) + (32) (n_{(-1,), up}) @ (n_{(0,), dn}) + (32) (n_{(0,), up}) @ (n_{(-1,), dn}) + (-40) (n_{(0,), up}) @ (n_{(0,), dn})


In [6]:
commute_list2 = [h1,h2,h1,h1,h1]
temp_result = commute_list2[0]
for i in range(1,len(commute_list2)):
    temp_result = fhc.commutator(commute_list2[i],temp_result)
print(fhc.simplify(temp_result))

(-32) (g_{(-1,), (0,), up}) @ (g_{(-1,), (0,), dn}) + (-24) (n_{(-1,), up}) @ (n_{(-1,), dn}) + (24) (n_{(-1,), up}) @ (n_{(0,), dn}) + (8) (n_{(-1,), up}) @ (Mn_{(0,), dn}) + (24) (n_{(0,), up}) @ (n_{(-1,), dn}) + (-24) (n_{(0,), up}) @ (n_{(0,), dn}) + (-8) (n_{(0,), up}) @ (Mn_{(0,), dn}) + (8) (n_{(-1,), dn}) @ (Mn_{(0,), up}) + (-8) (n_{(0,), dn}) @ (Mn_{(0,), up})


In [7]:
h1_shift = fhc.SumOp([fhc.HoppingOp((1,), ( 2,), s, v) for s in [0, 1]])
commute_list1 = [h1_shift,h2_origin,h1_shift,h1_shift,h1_shift]
temp_result = commute_list1[0]
for i in range(1,len(commute_list1)):
    temp_result = fhc.commutator(commute_list1[i],temp_result)
print(fhc.simplify(temp_result))

(-32) (g_{(1,), (2,), up}) @ (g_{(1,), (2,), dn}) + (-40) (n_{(1,), up}) @ (n_{(1,), dn}) + (32) (n_{(1,), up}) @ (n_{(2,), dn}) + (32) (n_{(2,), up}) @ (n_{(1,), dn}) + (-24) (n_{(2,), up}) @ (n_{(2,), dn})


In [8]:
commute_list2 = [h1_shift,h2,h1_shift,h1_shift,h1_shift]
temp_result = commute_list2[0]
for i in range(1,len(commute_list2)):
    temp_result = fhc.commutator(commute_list2[i],temp_result)
print(fhc.simplify(temp_result))

(-32) (g_{(1,), (2,), up}) @ (g_{(1,), (2,), dn}) + (-24) (n_{(1,), up}) @ (n_{(1,), dn}) + (24) (n_{(1,), up}) @ (n_{(2,), dn}) + (-8) (n_{(1,), up}) @ (Mn_{(1,), dn}) + (24) (n_{(2,), up}) @ (n_{(1,), dn}) + (-24) (n_{(2,), up}) @ (n_{(2,), dn}) + (8) (n_{(2,), up}) @ (Mn_{(1,), dn}) + (-8) (n_{(1,), dn}) @ (Mn_{(1,), up}) + (8) (n_{(2,), dn}) @ (Mn_{(1,), up})


In [20]:
print("err_bound:", err_bound)

err_bound: [2.6809447650447145, 17.50853678847553, 4.797982966781246, 0.8002940583523714, 0.12001862556075223]


In [21]:
print(sum(err_bound_origin))

26.06384787779088


In [22]:
print(sum(err_bound))

25.907777204214618


In [4]:
hlist = [h0,h1,h2]
translatt = fhc.SubLattice([[2]])
comm_tab2 = fhc.NestedCommutatorTable(hlist, 3, translatt)
tab2 = comm_tab2.table(2)
err_bound = 5 * [0]
"""
method = fhc.SplittingMethod.suzuki(len(hlist), 1)
s = (method.num_layers + 1) // 2
comm_bound_terms = fhc.commutator_bound(method, s)
"""
for term in comm_bound_terms:
    num_int=0
    for i in term.commidx:
        if i==3:
            num_int+=3
        elif i==2:
            num_int+=1
    err_bound[num_int] += term.weight * tab2[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()
    print(term)
    print(err_bound)

1/24 * [H_0, [H_1, H_0]]
[0.3333333333333333, 0, 0, 0, 0]
1/12 * [H_1, [H_1, H_0]]
[1.0, 0, 0, 0, 0]
1/12 * [H_2, [H_1, H_0]]
[1.0, 0.3333333333333333, 0, 0, 0]
1/24 * [H_0, [H_2, H_0]]
[1.0, 0.6666666666666666, 0, 0, 0]
1/12 * [H_1, [H_2, H_0]]
[1.0, 1.0, 0, 0, 0]
1/12 * [H_2, [H_2, H_0]]
[1.0, 1.0, 0.16666666666666666, 0, 0]
1/24 * [H_1, [H_2, H_1]]
[1.0, 1.3333333333333333, 0.16666666666666666, 0, 0]
1/12 * [H_2, [H_2, H_1]]
[1.0, 1.3333333333333333, 0.3333333333333333, 0, 0]


In [5]:
print(tab2[1][1][0])

<zero op>


## The Hamiltonian for the 2D square lattice
firstly,to construct the Hamiltonian terms

In [2]:
v=-1
u=1
translatt = fhc.SubLattice(np.array([[2, 0], [0, 2]]))
plaqcoords = [( 0,  0), ( 1,  0), ( 1,  1), ( 0,  1)]
plaqcshift = [(-1, -1), ( 0, -1), ( 0,  0), (-1,  0)]
h0 = fhc.SumOp([fhc.HoppingOp(plaqcoords[i], plaqcoords[(i + 1) % 4], s, v) for s in [0, 1] for i in range(4)])
h1 = fhc.SumOp([fhc.HoppingOp(plaqcshift[i], plaqcshift[(i + 1) % 4], s, v) for s in [0, 1] for i in range(4)])
h2 = fhc.SumOp([fhc.ProductOp([fhc.ModifiedNumOp(plaqcoords[i], s, 1) for s in [0, 1]], u) for i in range(4)])
hlist = [h0, h1, h2]
h2_origin = fhc.SumOp([fhc.ProductOp([fhc.NumberOp(plaqcoords[i], s, 1) for s in [0, 1]], u) for i in range(4)])
hlist_origin = [h0,h1,h2_origin]

To compute the error bound for Modified Hamiltonian in 2nd order Suzuki

In [ ]:
comm_tab = fhc.NestedCommutatorTable(hlist, 3, translatt)

tab2 = comm_tab.table(2)

method = fhc.SplittingMethod.suzuki(len(hlist), 1)
s = (method.num_layers + 1) // 2
comm_bound_terms = fhc.commutator_bound_strang(len(hlist))
err_bound = (method.order + 1) * [0]

for term in comm_bound_terms:
    num_int = sum(1 if i == len(hlist)-1 else 0 for i in term.commidx)
    err_bound[num_int] += term.weight * tab2[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()

In [3]:
print(err_bound)

[2.9428090415820636, 4.884565568152499, 0.8707908118985986]


To compute the error bound for Original Hamiltonian in 2nd order Suzuki (Verify the paper)

In [5]:
comm_tab_origin = fhc.NestedCommutatorTable(hlist_origin, 3, translatt)
tab2_origin = comm_tab_origin.table(2)
method = fhc.SplittingMethod.suzuki(len(hlist_origin), 1)
s = (method.num_layers + 1) // 2
comm_bound_terms_origin = fhc.commutator_bound_strang(len(hlist_origin))
err_bound_origin = (method.order + 1) * [0]

for term in comm_bound_terms_origin:
    num_int = sum(1 if i == len(hlist)-1 else 0 for i in term.commidx)
    err_bound_origin[num_int] += term.weight * tab2_origin[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()

Access to E.q.44, at Page 7. The result below satisfy: $r/4=c/6$

In [6]:
print(err_bound_origin)

[2.9428090415820636, 5.3925667105067046, 0.8707908118985986]


Modified Hamiltonian for higher order Suzuki (To be computed)

In [4]:
comm_tab = fhc.NestedCommutatorTable(hlist, 5, translatt)
tab4 = comm_tab.table(4)
method = fhc.SplittingMethod.suzuki(len(hlist), 2)
s = (method.num_layers + 1) // 2
comm_bound_terms = fhc.commutator_bound(method, s)
err_bound = (method.order + 1) * [0]
for term in comm_bound_terms:
    num_int = sum(1 if i == len(hlist)-1 else 0 for i in term.commidx)
    err_bound[num_int] += term.weight * tab4[term.commidx[0]][term.commidx[1]][term.commidx[2]][term.commidx[3]][term.commidx[4]].norm_bound()

In [5]:
print(err_bound)

[8.594092836201444, 314.6822436989095, 48.86064864644625, 3.9068044102160666, 0.3175290107048813]


Original Hamiltonian for higher order Suzuki (To be computed)

In [4]:
comm_tab_origin = fhc.NestedCommutatorTable(hlist_origin, 5, translatt)
tab4_origin = comm_tab_origin.table(4)
method = fhc.SplittingMethod.suzuki(len(hlist_origin), 2)
s = (method.num_layers + 1) // 2
comm_bound_terms_origin = fhc.commutator_bound(method, s)
err_bound_origin = (method.order + 1) * [0]
for term in comm_bound_terms_origin:
    num_int = sum(1 if i == len(hlist_origin)-1 else 0 for i in term.commidx)
    err_bound_origin[num_int] += term.weight * tab4_origin[term.commidx[0]][term.commidx[1]][term.commidx[2]][term.commidx[3]][term.commidx[4]].norm_bound()

In [5]:
print(err_bound_origin)

[8.594092836201444, 368.65698470191654, 57.37811329514797, 4.284967417220382, 0.3175290107048814]


## The Hamiltonian of 2D triangle lattice
Firstly, construct the hopping terms and useful variables.

In [2]:
from fractions import Fraction
v = -1
u = 1
translatt = fhc.SubLattice(np.array([[3, 0, -3], [0, 3, -3]]).T)
hexcoords = [( 2, -1, -1),
            ( 1,  1, -2),
            (-1,  2, -1),
            (-2,  1,  1),
            (-1, -1,  2),
            ( 1, -2,  1)]
hk = [fhc.SumOp([fhc.HoppingOp(( 0,  0,  0), hexcoords[i], s, v) for s in [0, 1]] +
                [fhc.HoppingOp(( 0,  0,  0), hexcoords[(i + 1) % 6], s, v) for s in [0, 1]] +
                [fhc.HoppingOp(hexcoords[i], hexcoords[(i + 1) % 6], s, v) for s in [0, 1]]) for i in range(0, 6, 2)]
hu_origin = fhc.SumOp([fhc.ProductOp([fhc.NumberOp(( 0,  0,  0), s, 1) for s in [0, 1]], u)] +
                [fhc.ProductOp([fhc.NumberOp(hexcoords[i], s, 1) for s in [0, 1]], Fraction(u, 3)) for i in range(6)])
hu = fhc.SumOp([fhc.ProductOp([fhc.ModifiedNumOp(( 0,  0,  0), s, 1) for s in [0, 1]], u)] +
                [fhc.ProductOp([fhc.ModifiedNumOp(hexcoords[i], s, 1) for s in [0, 1]], Fraction(u, 3)) for i in range(6)])
hlist = hk + [hu]
hlist_origin = hk + [hu_origin]
comm_tab = fhc.NestedCommutatorTable(hlist, 5, translatt)
tab2 = comm_tab.table(2)
tab4 = comm_tab.table(4)
comm_tab_origin = fhc.NestedCommutatorTable(hlist_origin, 5, translatt)
tab2_origin = comm_tab_origin.table(2)
tab4_origin = comm_tab_origin.table(4)

In [3]:
for methname in ["Strang", "Suzuki4"]:
    print(80 * "_")
    print("Method:", methname)
    if methname == "Strang":
        # Strang splitting method
        method = fhc.SplittingMethod.suzuki(len(hlist), 1)
        print(method)
        # tight bound
        print("bound specialized for second-order Suzuki method:")
        comm_bound_terms = fhc.commutator_bound_strang(len(hlist))
    elif methname == "Suzuki4":
        method = fhc.SplittingMethod.suzuki(len(hlist), 2)
        print(method)
        s = (method.num_layers + 1) // 2
        print("s:", s)
        comm_bound_terms = fhc.commutator_bound(method, s)
    # sort by number of interaction terms
    err_bound = (method.order + 1) * [0]
    for term in comm_bound_terms:
        print(term)
        num_int = sum(1 if i == len(hlist)-1 else 0 for i in term.commidx)
        if method.order == 2:
            err_bound[num_int] += term.weight * tab2[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()
        elif method.order == 4:
            err_bound[num_int] += term.weight * tab4[term.commidx[0]][term.commidx[1]][term.commidx[2]][term.commidx[3]][term.commidx[4]].norm_bound()
    print("err_bound:", err_bound)

________________________________________________________________________________
Method: Strang
Splitting method of order 2 for 4 terms using 7 layers,
  indices: [0, 1, 2, 3, 2, 1, 0]
  coeffs:  [0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5]
bound specialized for second-order Suzuki method:
1/24 * [H_0, [H_1, H_0]]
1/12 * [H_1, [H_1, H_0]]
1/12 * [H_2, [H_1, H_0]]
1/12 * [H_3, [H_1, H_0]]
1/24 * [H_0, [H_2, H_0]]
1/12 * [H_1, [H_2, H_0]]
1/12 * [H_2, [H_2, H_0]]
1/12 * [H_3, [H_2, H_0]]
1/24 * [H_0, [H_3, H_0]]
1/12 * [H_1, [H_3, H_0]]
1/12 * [H_2, [H_3, H_0]]
1/12 * [H_3, [H_3, H_0]]
1/24 * [H_1, [H_2, H_1]]
1/12 * [H_2, [H_2, H_1]]
1/12 * [H_3, [H_2, H_1]]
1/24 * [H_1, [H_3, H_1]]
1/12 * [H_2, [H_3, H_1]]
1/12 * [H_3, [H_3, H_1]]
1/24 * [H_2, [H_3, H_2]]
1/12 * [H_3, [H_3, H_2]]
err_bound: [19.736055087812034, 9.182587779763468, 0.9772838841927127]
________________________________________________________________________________
Method: Suzuki4
Splitting method of order 4 for 4 terms using 31 l

0.034834608076936585 * [H_2, [H_1, [H_3, [H_2, H_0]]]]
0.019436425463225913 * [H_3, [H_1, [H_3, [H_2, H_0]]]]
0.00815295566434147 * [H_0, [H_2, [H_3, [H_2, H_0]]]]
0.02586263566433723 * [H_1, [H_2, [H_3, [H_2, H_0]]]]
0.030348621870636906 * [H_2, [H_2, [H_3, [H_2, H_0]]]]
0.019436425463225913 * [H_3, [H_2, [H_3, [H_2, H_0]]]]
0.008152955664341472 * [H_0, [H_3, [H_3, [H_2, H_0]]]]
0.02586263566433723 * [H_1, [H_3, [H_3, [H_2, H_0]]]]
0.02586263566433723 * [H_2, [H_3, [H_3, [H_2, H_0]]]]
0.030785555408577317 * [H_3, [H_3, [H_3, [H_2, H_0]]]]
0.004701334310169882 * [H_0, [H_0, [H_0, [H_3, H_0]]]]
0.015024679162295492 * [H_1, [H_0, [H_0, [H_3, H_0]]]]
0.015024679162295492 * [H_2, [H_0, [H_0, [H_3, H_0]]]]
0.009689668290122168 * [H_3, [H_0, [H_0, [H_3, H_0]]]]
0.00463891008158979 * [H_0, [H_1, [H_0, [H_3, H_0]]]]
0.023863964287831545 * [H_1, [H_1, [H_0, [H_3, H_0]]]]
0.029806758494076466 * [H_2, [H_1, [H_0, [H_3, H_0]]]]
0.017328153057109535 * [H_3, [H_1, [H_0, [H_3, H_0]]]]
0.0046389100815

0.0034675615540058972 * [H_0, [H_3, [H_0, [H_3, H_1]]]]
0.015274014887336403 * [H_1, [H_3, [H_0, [H_3, H_1]]]]
0.015274014887336403 * [H_2, [H_3, [H_0, [H_3, H_1]]]]
0.016491847443666083 * [H_3, [H_3, [H_0, [H_3, H_1]]]]
0.005703818762629349 * [H_0, [H_0, [H_1, [H_3, H_1]]]]
0.016093031635594276 * [H_1, [H_0, [H_1, [H_3, H_1]]]]
0.016093031635594276 * [H_2, [H_0, [H_1, [H_3, H_1]]]]
0.010496866451826755 * [H_3, [H_0, [H_1, [H_3, H_1]]]]
0.006200708118368314 * [H_0, [H_1, [H_1, [H_3, H_1]]]]
0.028985922257517838 * [H_1, [H_1, [H_1, [H_3, H_1]]]]
0.03966797226237877 * [H_2, [H_1, [H_1, [H_3, H_1]]]]
0.02559273106643771 * [H_3, [H_1, [H_1, [H_3, H_1]]]]
0.004638910081589789 * [H_0, [H_2, [H_1, [H_3, H_1]]]]
0.017921170081586616 * [H_1, [H_2, [H_1, [H_3, H_1]]]]
0.031847670551676116 * [H_2, [H_2, [H_1, [H_3, H_1]]]]
0.03147026813183503 * [H_3, [H_2, [H_1, [H_3, H_1]]]]
0.00463891008158979 * [H_0, [H_3, [H_1, [H_3, H_1]]]]
0.017921170081586616 * [H_1, [H_3, [H_1, [H_3, H_1]]]]
0.01792117008

In [4]:
for methname in ["Strang", "Suzuki4"]:
    print(80 * "_")
    print("Method:", methname)
    if methname == "Strang":
        # Strang splitting method
        method = fhc.SplittingMethod.suzuki(len(hlist_origin), 1)
        print(method)
        # tight bound
        print("bound specialized for second-order Suzuki method:")
        comm_bound_terms = fhc.commutator_bound_strang(len(hlist_origin))
    elif methname == "Suzuki4":
        method = fhc.SplittingMethod.suzuki(len(hlist_origin), 2)
        print(method)
        s = (method.num_layers + 1) // 2
        print("s:", s)
        comm_bound_terms = fhc.commutator_bound(method, s)
    # sort by number of interaction terms
    err_bound = (method.order + 1) * [0]
    for term in comm_bound_terms:
        print(term)
        num_int = sum(1 if i == len(hlist)-1 else 0 for i in term.commidx)
        if method.order == 2:
            err_bound[num_int] += term.weight * tab2_origin[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()
        elif method.order == 4:
            err_bound[num_int] += term.weight * tab4_origin[term.commidx[0]][term.commidx[1]][term.commidx[2]][term.commidx[3]][term.commidx[4]].norm_bound()
    print("err_bound:", err_bound)

________________________________________________________________________________
Method: Strang
Splitting method of order 2 for 4 terms using 7 layers,
  indices: [0, 1, 2, 3, 2, 1, 0]
  coeffs:  [0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5]
bound specialized for second-order Suzuki method:
1/24 * [H_0, [H_1, H_0]]
1/12 * [H_1, [H_1, H_0]]
1/12 * [H_2, [H_1, H_0]]
1/12 * [H_3, [H_1, H_0]]
1/24 * [H_0, [H_2, H_0]]
1/12 * [H_1, [H_2, H_0]]
1/12 * [H_2, [H_2, H_0]]
1/12 * [H_3, [H_2, H_0]]
1/24 * [H_0, [H_3, H_0]]
1/12 * [H_1, [H_3, H_0]]
1/12 * [H_2, [H_3, H_0]]
1/12 * [H_3, [H_3, H_0]]
1/24 * [H_1, [H_2, H_1]]
1/12 * [H_2, [H_2, H_1]]
1/12 * [H_3, [H_2, H_1]]
1/24 * [H_1, [H_3, H_1]]
1/12 * [H_2, [H_3, H_1]]
1/12 * [H_3, [H_3, H_1]]
1/24 * [H_2, [H_3, H_2]]
1/12 * [H_3, [H_3, H_2]]
err_bound: [19.736055087812034, 10.079711868705227, 0.9772838841927125]
________________________________________________________________________________
Method: Suzuki4
Splitting method of order 4 for 4 terms using 31 

0.034834608076936585 * [H_2, [H_1, [H_3, [H_2, H_0]]]]
0.019436425463225913 * [H_3, [H_1, [H_3, [H_2, H_0]]]]
0.00815295566434147 * [H_0, [H_2, [H_3, [H_2, H_0]]]]
0.02586263566433723 * [H_1, [H_2, [H_3, [H_2, H_0]]]]
0.030348621870636906 * [H_2, [H_2, [H_3, [H_2, H_0]]]]
0.019436425463225913 * [H_3, [H_2, [H_3, [H_2, H_0]]]]
0.008152955664341472 * [H_0, [H_3, [H_3, [H_2, H_0]]]]
0.02586263566433723 * [H_1, [H_3, [H_3, [H_2, H_0]]]]
0.02586263566433723 * [H_2, [H_3, [H_3, [H_2, H_0]]]]
0.030785555408577317 * [H_3, [H_3, [H_3, [H_2, H_0]]]]
0.004701334310169882 * [H_0, [H_0, [H_0, [H_3, H_0]]]]
0.015024679162295492 * [H_1, [H_0, [H_0, [H_3, H_0]]]]
0.015024679162295492 * [H_2, [H_0, [H_0, [H_3, H_0]]]]
0.009689668290122168 * [H_3, [H_0, [H_0, [H_3, H_0]]]]
0.00463891008158979 * [H_0, [H_1, [H_0, [H_3, H_0]]]]
0.023863964287831545 * [H_1, [H_1, [H_0, [H_3, H_0]]]]
0.029806758494076466 * [H_2, [H_1, [H_0, [H_3, H_0]]]]
0.017328153057109535 * [H_3, [H_1, [H_0, [H_3, H_0]]]]
0.0046389100815

0.0034675615540058972 * [H_0, [H_3, [H_0, [H_3, H_1]]]]
0.015274014887336403 * [H_1, [H_3, [H_0, [H_3, H_1]]]]
0.015274014887336403 * [H_2, [H_3, [H_0, [H_3, H_1]]]]
0.016491847443666083 * [H_3, [H_3, [H_0, [H_3, H_1]]]]
0.005703818762629349 * [H_0, [H_0, [H_1, [H_3, H_1]]]]
0.016093031635594276 * [H_1, [H_0, [H_1, [H_3, H_1]]]]
0.016093031635594276 * [H_2, [H_0, [H_1, [H_3, H_1]]]]
0.010496866451826755 * [H_3, [H_0, [H_1, [H_3, H_1]]]]
0.006200708118368314 * [H_0, [H_1, [H_1, [H_3, H_1]]]]
0.028985922257517838 * [H_1, [H_1, [H_1, [H_3, H_1]]]]
0.03966797226237877 * [H_2, [H_1, [H_1, [H_3, H_1]]]]
0.02559273106643771 * [H_3, [H_1, [H_1, [H_3, H_1]]]]
0.004638910081589789 * [H_0, [H_2, [H_1, [H_3, H_1]]]]
0.017921170081586616 * [H_1, [H_2, [H_1, [H_3, H_1]]]]
0.031847670551676116 * [H_2, [H_2, [H_1, [H_3, H_1]]]]
0.03147026813183503 * [H_3, [H_2, [H_1, [H_3, H_1]]]]
0.00463891008158979 * [H_0, [H_3, [H_1, [H_3, H_1]]]]
0.017921170081586616 * [H_1, [H_3, [H_1, [H_3, H_1]]]]
0.01792117008

## 2D square lattice in H&V classification

In [4]:
v=-1
u=1
translatt = fhc.SubLattice(np.array([[2, 0], [0, 2]]))
hlist = []
ph_even = [[( 0,  0), ( 1,  0)],[(0, -1),(-1, -1)]]
ph_odd = [[( 0,  0), (-1,  0)],[( 0,  1), ( 1,  1)]]
pv_even = [[( 0,  0), (1,  0)],[(-1, -1), (-1,  0)]]
pv_odd = [[( 0,  0), (-1,  0)],[(1,  0), ( 1,  1)]]
plaqcoords = [( 0,  0), ( 1,  0), ( 1,  1), ( 0,  1)]
bound_list = [ph_even, ph_odd, pv_even, pv_odd]
for bound in bound_list:
    hlist.append(fhc.SumOp([fhc.HoppingOp(bound[i][0], bound[i][1], s, v) for s in [0,1] for i in [0,1]]))
h_inter = fhc.SumOp([fhc.ProductOp([fhc.ModifiedNumOp(plaqcoords[i], s, 1) for s in [0, 1]], u) for i in range(4)])
hlist_origin = hlist.copy()
hlist.append(h_inter)
h_inter_origin = fhc.SumOp([fhc.ProductOp([fhc.NumberOp(plaqcoords[i], s, 1) for s in [0, 1]], u) for i in range(4)])
hlist_origin.append(h_inter_origin)

In [5]:
comm_tab = fhc.NestedCommutatorTable(hlist, 3, translatt)

tab2 = comm_tab.table(2)

method = fhc.SplittingMethod.suzuki(len(hlist), 1)
s = (method.num_layers + 1) // 2
comm_bound_terms = fhc.commutator_bound_strang(len(hlist))
err_bound = (method.order + 1) * [0]

for term in comm_bound_terms:
    num_int = sum(1 if i == len(hlist)-1 else 0 for i in term.commidx)
    err_bound[num_int] += term.weight * tab2[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()

In [6]:
print(err_bound)

[16.232918688748597, 9.544519951451555, 1.2041241452319313]


In [7]:
comm_tab_origin = fhc.NestedCommutatorTable(hlist_origin, 3, translatt)
tab2_origin = comm_tab_origin.table(2)
method = fhc.SplittingMethod.suzuki(len(hlist_origin), 1)
s = (method.num_layers + 1) // 2
comm_bound_terms_origin = fhc.commutator_bound_strang(len(hlist_origin))
err_bound_origin = (method.order + 1) * [0]

for term in comm_bound_terms_origin:
    num_int = sum(1 if i == len(hlist)-1 else 0 for i in term.commidx)
    err_bound_origin[num_int] += term.weight * tab2_origin[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()

In [8]:
print(err_bound_origin)

[16.232918688748597, 9.78677964310814, 1.2041241452319313]
